In [1]:
import pickle
import pandas as pd
import numpy as np
import arrow
import pprint
from collections import Counter
from datetime import datetime
from tqdm import tqdm

In [2]:
posts = pickle.load(open('rateme_submissions.dat', 'rb'))
print(len(posts))

28893


In [11]:
# post = posts[0]
# comment = post.comments[0]

In [18]:
# for key in vars(post).keys():
#     print(vars(post)[key])

In [19]:
# posts_df = pd.DataFrame(posts)

In [3]:
posts_vars = [vars(post) for post in posts]

## Posts exploration

In [111]:
lists_of_keys = [list(post_vars.keys()) for post_vars in posts_vars]

In [112]:
keys = sorted(list(set([item for sublist in lists_of_keys for item in sublist])))

In [113]:
key_values = {key : [post_vars[key] if key in post_vars
                     else None for post_vars in posts_vars] for key in keys}

In [114]:
list(enumerate(keys))

[(0, '_comments'),
 (1, '_comments_by_id'),
 (2, '_fetched'),
 (3, '_flair'),
 (4, '_info_params'),
 (5, '_mod'),
 (6, '_reddit'),
 (7, 'approved_at_utc'),
 (8, 'approved_by'),
 (9, 'archived'),
 (10, 'author'),
 (11, 'author_cakeday'),
 (12, 'author_flair_css_class'),
 (13, 'author_flair_text'),
 (14, 'banned_at_utc'),
 (15, 'banned_by'),
 (16, 'brand_safe'),
 (17, 'can_gild'),
 (18, 'can_mod_post'),
 (19, 'clicked'),
 (20, 'comment_limit'),
 (21, 'comment_sort'),
 (22, 'contest_mode'),
 (23, 'created'),
 (24, 'created_utc'),
 (25, 'distinguished'),
 (26, 'domain'),
 (27, 'downs'),
 (28, 'edited'),
 (29, 'gilded'),
 (30, 'hidden'),
 (31, 'hide_score'),
 (32, 'id'),
 (33, 'is_crosspostable'),
 (34, 'is_reddit_media_domain'),
 (35, 'is_self'),
 (36, 'is_video'),
 (37, 'likes'),
 (38, 'link_flair_css_class'),
 (39, 'link_flair_text'),
 (40, 'locked'),
 (41, 'media'),
 (42, 'media_embed'),
 (43, 'mod_note'),
 (44, 'mod_reason_by'),
 (45, 'mod_reason_title'),
 (46, 'mod_reports'),
 (47, 'n

In [345]:
Counter(key_values[keys[83]]).most_common()

[('house_only', 28893)]

In [241]:
medias = list(filter(lambda x: x is not None, key_values[keys[41]]))

In [341]:
indices = [i for i, x in enumerate(key_values[keys[80]]) if x is not None]

In [342]:
print(indices[:5])

[0, 1, 2, 3, 4]


In [324]:
# [x for x in key_values['media'] if x is not None and
#                                    'oembed' in x.keys() and
#                                    'url' in x['oembed'].keys() and
#                                    'youtube' in x['oembed']['url'].lower()]

In [312]:
# [post.title for post in [posts[index] for index in indices]]

## Create DF

In [5]:
df = pd.DataFrame()

In [421]:
post = posts[1000]
row = {}

In [422]:
if post.distinguished == 'moderator':
#     break
    pass
if post.link_flair_text is not None:
    # break
    pass

In [423]:
row['author'] = post.author.name
row['author_has_flair'] = (post.author_flair_text is not None)
row['created'] = pd.Timestamp(datetime.utcfromtimestamp(post.created_utc))
row['edited'] = (post.edited != False)
row['gilded'] = (post.gilded == 1)
row['id'] = post.id
row['is_self'] = post.is_self
row['over_18'] = post.over_18
row['permalink'] = post.permalink
row['score'] = post.score
row['selftext'] = post.selftext
row['title'] = post.title

if post.media is not None and 'oembed' in post.media.keys():
    row['width'] = post.media['oembed']['width']
    row['height'] = post.media['oembed']['height']
    row['thumbnail_width'] = post.media['oembed']['thumbnail_width']
    row['thumbnail_height'] = post.media['oembed']['thumbnail_height']
else:
    row['width'] = row['height'] = row['thumbnail_width'] = row['thumbnail_height'] = None

In [425]:
# row

In [435]:
df = df.append([row])

In [6]:
df = pd.DataFrame()
for post in tqdm(posts):
    row = {}
    if post.distinguished == 'moderator':
        continue
    if post.link_flair_text is not None:
        continue
        
    row['author'] = post.author.name
    row['author_has_flair'] = (post.author_flair_text is not None)
    row['created'] = pd.to_datetime(datetime.utcfromtimestamp(post.created_utc))
    row['edited'] = (post.edited != False)
    row['gilded'] = (post.gilded == 1)
    row['id'] = post.id
    row['is_self'] = post.is_self
    row['over_18'] = post.over_18
    row['permalink'] = post.permalink
    row['score'] = post.score
    row['selftext'] = post.selftext
    row['title'] = post.title

    if post.media is not None and 'oembed' in post.media.keys():
        if 'width' in post.media['oembed'].keys():
            row['width'] = post.media['oembed']['width']
        if 'height' in post.media['oembed'].keys():
            row['height'] = post.media['oembed']['height']
        if 'thumbnail_width' in post.media['oembed'].keys():
            row['thumbnail_width'] = post.media['oembed']['thumbnail_width']
        if 'thumbnail_height' in post.media['oembed'].keys():
            row['thumbnail_height'] = post.media['oembed']['thumbnail_height']
    else:
        row['width'] = row['height'] = row['thumbnail_width'] = row['thumbnail_height'] = None
        
    df = df.append([row])

100%|████████████████████████████████████████████████████████████████████████████| 28893/28893 [12:23<00:00, 24.45it/s]


## To/from CSV. Failed for some obscure reason. I'll pickle.

In [195]:
df.to_csv('df.csv', encoding='utf8', index=False)

In [34]:
df.head()

,author,author_has_flair,created,edited,gilded,height,id,is_self,over_18,permalink,score,selftext,thumbnail_height,thumbnail_width,title,width
0,djantigravity,False,2009-12-15 06:02:36,False,False,None,aes21,True,False,/r/Rateme/comments/aes21/nervous_but_ill_give_...,7,http://imgur.com/d12Cd Me a couple of months a...,None,None,"Nervous, but I'll give it a shot. [M]",None
0,hitogokoro,False,2009-12-15 05:53:59,True,False,None,aeryj,True,False,/r/Rateme/comments/aeryj/i_posted_pictures_in_...,8,So I'm 21 now and a college student. I thought...,None,None,"I posted pictures in a couple of other posts, ...",None
0,nictheamazing,False,2010-04-26 23:04:41,False,False,None,bwg68,False,False,/r/Rateme/comments/bwg68/rate/,29,,None,None,rate?,None
0,zoide,False,2010-07-02 03:06:16,False,False,None,cl62k,False,False,/r/Rateme/comments/cl62k/after_a_long_day_and_...,31,,None,None,after a long day and a lot of weed,None
0,yourpasswordisshit,False,2010-07-04 04:37:24,False,False,None,clsmq,False,False,/r/Rateme/comments/clsmq/me/,3,,None,None,Me,None


In [176]:
df.shape

(28845, 16)

In [179]:
df.dtypes

author                      object
author_has_flair              bool
created             datetime64[ns]
edited                        bool
gilded                        bool
height                      object
id                          object
is_self                       bool
over_18                       bool
permalink                   object
score                        int64
selftext                    object
thumbnail_height            object
thumbnail_width             object
title                       object
width                       object
dtype: object

In [168]:
new_df.dtypes

author                      object
author_has_flair              bool
created             datetime64[ns]
edited                      object
gilded                      object
height                     float64
id                          object
is_self                     object
over_18                     object
permalink                   object
score                      float64
selftext                    object
thumbnail_height           float64
thumbnail_width            float64
title                       object
width                      float64
dtype: object

In [196]:
new_df = pd.read_csv('df.csv')

In [199]:
df.shape

(28845, 16)

In [201]:
# new_df[new_df.created.isnull()]

In [226]:
df.to_csv('df.csv', encoding='utf8', index=False)

In [227]:
def date_parser(time_series):
    return pd.to_datetime(time_series, errors='coerce')

new_df = pd.read_csv('df.csv', encoding = 'utf8', index_col=None,
                    dtype = {'created': str, 'author_has_flair': bool}, parse_dates=['created'], date_parser=date_parser)

# new_df = new_df.dropna(subset=['created'])
# new_df.author_has_flair = new_df.author_has_flair.astype(bool)
# new_df.edited = new_df.edited.astype(bool) # messy!
# new_df.author_has_flair = new_df.author_has_flair.astype(bool)
# new_df.author_has_flair = new_df.author_has_flair.astype(bool)
# new_df.author_has_flair = new_df.author_has_flair.astype(bool)

In [225]:
new_df.shape

(28865, 16)

In [171]:
Counter(new_df.author_has_flair)

Counter({False: 28561, True: 284, nan: 17})

## Pickling

In [231]:
pickle.dump(df, open('posts_df.dat', 'wb'))

In [8]:
posts_df = pickle.load(open('posts_df.dat', 'rb'))

## Comments

In [242]:
post = posts[1000]

In [243]:
post.id

'u35ps'

In [251]:
# df[df.id.str.contains(post.id)]

In [252]:
comments = post.comments.list()

In [259]:
# indices = np.random.randint(0, 10000, 100)
# for index in tqdm(indices):
#     post = posts[index]
#     comments = post.comments.list()
#     for comment in comments:
#         if comment.downs > 0:
#             print(comment.downs)

author, body, ups, id, permalink, created_utc, edited, gilded, parent_id (remove first 3 letters), is_submitter, replies

In [287]:
comment = comments[3]

In [288]:
comment.parent_id

't1_ccd5kbh'

In [10]:
df = pd.DataFrame()
for i, post in tqdm(enumerate(posts)):
    if not posts_df.id.str.contains(post.id).any():
        continue
    comments_forest = post.comments
    replacement_res = comments_forest.replace_more()  # I don't need the return value
    comments = comments_forest.list()
    for comment in comments:
        if comment.author is None:
            continue
        row = {}
        row['name'] = comment.author.name
        row['body'] = comment.body
        row['score'] = comment.ups
        row['id'] = comment.id
        row['permalink'] = comment.permalink
        row['created'] = pd.to_datetime(datetime.utcfromtimestamp(comment.created_utc))
        row['edited'] = comment.edited
        row['gilded'] = (comment.gilded == 1)
        row['parent_post_id'] = comment.parent_id[3:] if comment.parent_id[:3] == 't3_' else None
        row['parent_comment_id'] = comment.parent_id[3:] if comment.parent_id[:3] == 't1_' else None
        if comment.parent_id[:3] != 't3_' and comment.parent_id[:3] != 't1_':
            pprint.pprint(vars(comment))
#         row['parent_id'] = comment.parent_id
        row['is_submitter'] = comment.is_submitter
        row['replies'] = [comment.id for comment in comment.replies.list()]
        
        df = df.append([row])
    if i % 1000 == 0:
        pickle.dump(df, open('comments_df.dat', 'wb'))

6893it [3:13:14,  4.93s/it]


In [12]:
first = pickle.load(open('comments_df_until_17000.dat', 'rb'))

In [13]:
second = pickle.load(open('comments_df_17000_22000.dat', 'rb'))

In [14]:
third = pickle.load(open('comments_df_22000plus.dat', 'rb'))

In [20]:
full = pd.concat([first, second, third])

In [21]:
pickle.dump(full, open('comments_df.dat', 'wb'))